In [18]:
import requests
import json
import pandas
import re
import time

# JSON

JavaScript Object Notation : JavaScript에서 객체를 표현하는 문법을 이용해 데이터를 표현하는 문서의 양식

정수 : 100

실수 : 11.11

참거짓 : true, false

문자열 : "문자열"

배열 : []   => 리스트

객체 : {}   => 딕셔너리









In [19]:
chk = False

In [20]:
def getKeyword() :
    keyword = input('검색어를 입력해 주세요 : ')
    return keyword

In [21]:
def connectionSite(keyword, start):
    # 요청할 페이지의 주소
    site = 'https://openapi.naver.com/v1/search/blog.json'
    # 파라미터 데이터
    params = {
        'query' : keyword,
        'start' : start,
        'display' : 100,
        'sort' : 'date'
    }
    # 헤더정보
    headers = {
        'X-Naver-Client-Id' : 'Fx7kgsRNPQ7XGU5LeQSE',
        'X-Naver-Client-Secret' : '2s01pestJ4'
    }
    # 요청한다.
    response = requests.get(site, params=params, headers=headers)
    # json 객체를 만든다(딕셔너리나 리스트로 만들어진다)
    json_dic = json.loads(response.text)
    # print(type(json_dic))
    return json_dic

In [22]:
# 전달된 문자열에서 html 태그를 제거한다.
def getText(data_str) :
    temp_str = re.sub('<.+?>', '', data_str, 0).strip()
    return temp_str

In [23]:
def getData(json_obj, keyword, start) :
    # print(item_list)
    # item 태그의 개수만큼 반복
    for item_tag in json_obj['items'] :
        # title 값 추출
        title_str = getText(item_tag['title'])
        # print(title_str)
        # link 추출
        link_str = getText(item_tag['link'])
        # print(link_str)
        # description 추출
        description_str = getText(item_tag['description'])
        # print(description_str)
        bloggername_str = getText(item_tag['bloggername'])
        bloggerlink_str = getText(item_tag['bloggerlink'])
        postdate_str = getText(item_tag['postdate'])

        data_dic = {
            'title' : [title_str],
            'link' : [link_str],
            'description' : [description_str],
            'bloggername' : [bloggername_str],
            'bloggerlink' : [bloggerlink_str],
            'postdate' : [postdate_str]
        }

        df = pandas.DataFrame(data_dic)

        global chk
        
        if chk == False :
            chk = True
            df.to_csv(f'{keyword}.csv', index=False, encoding='utf-8-sig')
        else :
            df.to_csv(f'{keyword}.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

        # total 값을 가져온다.
        total = json_obj['total']
        
    return total

In [24]:
start = 1

keyword = getKeyword()

while True :
    time.sleep(0.5)
    json_obj = connectionSite(keyword, start)
    total = getData(json_obj, keyword, start)
    
    print(f'{start} / {total}')
    
    # 더이상 있는지 확인한다.
    start = start + 100

    # 1000개 이하일 경우
    if start >= total :
        break
    # 901번까지 왔다면 1000으로 셋팅한다.(마지막 페이지)
    elif start == 1001 :
        start = 1000
    # 1000번까지 왔다면 종료
    elif start == 1100 :
        break

검색어를 입력해 주세요 : 캡틴마블
1 / 30884
101 / 30884
201 / 30884
301 / 30884
401 / 30884
501 / 30884
601 / 30884
701 / 30884
801 / 30884
901 / 30884
1000 / 30884
